
# Проверка гипотез на основе данных о рейтингах фильмов
_____________________________________________________


## Библиотеки

In [1]:
import pandas as pd  
import numpy as np
from scipy.stats import mannwhitneyu
from scipy import stats

#### Load Data

In [ ]:
data1 = pd.DataFrame(pd.read_csv('movieset.csv'))

In [7]:
data1

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,3.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,0,0
1093,3.0,4.0,NaN,NaN,4.0,4.0,2.5,NaN,3.5,3.5,...,5.0,3.0,5.0,5.0,5.0,6.0,5.0,1.0,0,0
1094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,6.0,3.0,1.0,6.0,6.0,4.0,2.0,1.0,0,0
1095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,4.0,3.0,3.0,1.0,1.0,0,1


#### Вопрос 1

#### Являются ли фильмы, которые более популярны (имеющие более высокие рейтинги), более рейтинговыми, чем фильмы, которые менее популярны?

- H0 (Нулевая): Рейтинг популярных фильмов не отличается от рейтинга непопулярных.
- H1: Рейтинг популярных фильмов отличается от рейтинга непопулярных.

In [ ]:
popularities = data1.count() #подсчет строк для рейтингов для определения популярности
median = popularities.median() #берем медиану популярности


low_pop = popularities[popularities < median] #Помещаем низкую популярность в массив
high_pop = popularities[popularities >= median] #Помещаем высокую популярность в массив

#Получаем рейтинги для обеих категорий
low_pop_ratings = data1[low_pop.index]
high_pop_ratings = data1[high_pop.index]

#Находим медианы для обеих категорий
med_low_pop = low_pop_ratings.median(skipna=True)
med_high_pop = high_pop_ratings.median(skipna=True)

#манн-Уитни U тест
mannwhitneyu(med_high_pop, med_low_pop, alternative='greater') #Проверка гипотезы о том, что медиана рейтингов популярных фильмов больше, чем у непопулярных

MannwhitneyuResult(statistic=45749.0, pvalue=3.418584551895982e-33)

#### Вывод:

Эта задача была решена с помощью одностороннего U-критерия Манна-Уитни, где критерий Манна-Уитни проверяет гипотезу относительно медианы генеральной совокупности. Критерий Манна-Уитни позволяет нам сравнивать категориальные переменные для измерения изменчивости, что согласуется с нашей целью – определить, имеют ли популярные фильмы такие же рейтинги, как и непопулярные. В данном случае я взял данные по популярности и разделил их по медиане, чтобы получить две равноценные выборки: с низкой популярностью и с высокой популярностью. Данные были очищены поэлементно, чтобы учесть только оценки участников. Благодаря поэлементной очистке мы сохраняем больше данных. Этот метод предотвращает потерю данных, поскольку очистка по строкам привела бы к уменьшению размера выборки. Затем была рассчитана медиана для каждой группы, чтобы обеспечить лучшее представление «типичного» значения. В результате моё p-значение оценивается как 3,418584551895982e-33, и при уровне альфа 0,005 мы отвергаем нулевую гипотезу и принимаем альтернативную гипотезу.

#### Вопрос 2:

#### Отличаются ли рейтинги новых фильмов от рейтингов старых?

- H0 (Нулевая): Рейтинг новых фильмов не отличается от рейтинга старых фильмов.
- H1: Рейтинг более новых фильмов  отличается от рейтинга старых фильмов.

In [9]:
movieratingandtitles = data1.iloc[:,:400] #берем первые 400 столбцов
movieTitles = pd.Series(list(movieratingandtitles)) #берем названия фильмов

# извлекаем годы
movieYears = movieratingandtitles.columns.str.extract('\((\d+)\)').astype(int)
# объединяем названия и годы в один массив
newmovieandyearcolumn= np.column_stack([movieTitles, movieYears])

newmovieandyearcolumn = newmovieandyearcolumn[(newmovieandyearcolumn[:, 1].argsort())]

#Разделение фильмов на новые и старые
oldermovie = movieratingandtitles.loc[:, pd.DataFrame(newmovieandyearcolumn[:200])[0].tolist()].median(axis=0)  #split into lower ratings
newermovie = movieratingandtitles.loc[:, pd.DataFrame(newmovieandyearcolumn[200:])[0].tolist()].median(axis=0) #split into higher ratings
#очистка от NaN
oldermovie_e = oldermovie[np.isfinite(oldermovie)]
newermovie_e = newermovie[np.isfinite(newermovie)]

mannwhitneyu(oldermovie_e, newermovie_e, alternative='two-sided') #Двусторонний тест


<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
C:\Users\Mi\AppData\Local\Temp\ipykernel_28768\1221330555.py:5: SyntaxWarning: invalid escape sequence '\('
  movieYears = movieratingandtitles.columns.str.extract('\((\d+)\)').astype(int)


MannwhitneyuResult(statistic=18009.5, pvalue=0.06974757637794209)

#### Вывод:

- Эта проблема была решена с помощью двухстороннего критерия Манна-Уитни, где тест проводился для определения, различаются ли рейтинги новых фильмов от рейтингов старых. Код был реализован путем поэлементной очистки данных. Опять же, это позволило соответствующим образом подсчитать данные, не влияя на размер данных. В этом наборе данных у одного из фильмов не был указан год. Вместо того, чтобы исключить фильм «Рэмбо: Первая кровь 2 (1985)» из набора данных, я ввел год, чтобы уменьшить вероятность увеличения стандартной ошибки. Информацию или данные следует удалять только тогда, когда мы уверены, что они не важны. В результате мое p-значение равно 0,0697475763779420. При уровне альфа 0,005 нам не удается отвергнуть нулевую гипотезу из-за случайности и принять альтернативную гипотезу.

#### Question #3:

#### Имеет ли значение гендерный аспект в оценке фильма «Шрек (2001)», т. е. оценивают ли его по-разному зрители-мужчины и зрители-женщины?

- H0 (Нулевая): Мужчины и женщины одинаково довольны «Шреком (2001)».
- H1: Мужчины и женщины по-разному довольны «Шреком (2001)».

In [10]:
title= 'Shrek' ## название фильма для анализа
data1.columns.get_loc('Shrek (2001)')

#Получение данных по гендерному признаку и рейтингу фильма Шрек(2001)
dataone = data1.iloc[:,[87,474]]

##Разделение данных по гендерному признаку
dataforfemale =  dataone.loc[dataone['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 1 ]
dataformale =  dataone.loc[dataone['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 2 ]

# разделение данных по гендерному признаку и преобразование в массивы numpy
dataformalenp = np.array(dataformale['Shrek (2001)'])
dataforfemalenp = np.array(dataforfemale['Shrek (2001)'])

#очистка от NaN
datafem = dataforfemalenp[np.isfinite(dataforfemalenp)]
datamal = dataformalenp[np.isfinite(dataformalenp)]

##Манн-Уитни U тест:
testman = mannwhitneyu(datafem, datamal, use_continuity=True, alternative='two-sided')

print(testman)


MannwhitneyuResult(statistic=96830.5, pvalue=0.050536625925559006)


#### Вывод:

Эта задача была решена с помощью двухстороннего критерия Манна-Уитни. Каждый интересующий нас образец был сохранен в массиве, соответствующем рейтингам фильма «Шрек (2001)». Затем данные были очищены поэлементно, чтобы размер выборки оставался больше, чем при очистке по строкам. Мы пришли к выводу, что мужчины и женщины воспринимают фильмы одинаково. В результате моё p-значение составило 0,0505366, а при уровне альфа 0,005 нам не удалось отвергнуть нулевое значение из-за случайности.

#### Вопрос 4:

#### Какая доля фильмов оценивается по-разному зрителями мужского и женского пола?

In [11]:
## Будем проводить тест Манна-Уитни для всех 400 фильмов, чтобы определить, сколько из них имеют значительные различия в рейтингах между мужчинами и женщинами.
all_titles = data1.columns
count = 0

# Проходим по всем 400 фильмам
for i in range(400):
    
    ## Устанавливаем столбцы для текущего фильма и гендерной идентичности
    dataone = data1.loc[:,[all_titles[i], 'Gender identity (1 = female; 2 = male; 3 = self-described)']]
    
    # Очистка данных от NaN
    dataone = dataone[np.isfinite(dataone)]
    
    # Разделение данных на женские и мужские
    datafemale =  dataone.loc[dataone['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 1 ].iloc[:,0]
    datamale =  dataone.loc[dataone['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 2 ].iloc[:,0]
    
    # Очистка от NaN
    datafem = datafemale[np.isfinite(datafemale)]
    datamal = datamale[np.isfinite(datamale)]
    
    # Для всех женщин и мужчин проводим тест Манна-Уитни, где для каждого p-значения выше 0.005 мы добавляем один к нашему счетчику
    test22, p = mannwhitneyu(datafem, datamal)
    
    if p < 0.005:
        count += 1
#Calculate the proportion based off our out count, out of our sample size of 400
proportion = count/400

print("Доля фильмов, оцененных зрителями-мужчинами и зрителями-женщинами по-разному, составляет ", count, "из 400, или", proportion, '.')


Доля фильмов, оцененных зрителями-мужчинами и зрителями-женщинами по-разному, составляет  50 из 400, или 0.125 .


#### Вывод:

- В этом вопросе я запустил двухсторонний тест Манна-Уитни, чтобы вычислить долю каждой подгруппы. В этом случае я очистил данные поэлементно, чтобы исключить возможность экспоненциального уменьшения размера выборки. Сначала я сгруппировал свои данные по двум категориям, мужчинам и женщинам, чтобы вычислить u-тест с точки зрения женщин и мужчин. С помощью цикла for для каждого значения выше 0,005 мы прибавляем единицу к нашему счету. Затем мы берем это число и делим его на общий размер выборки, чтобы получить долю мужчин и женщин, которые смотрят фильмы по-разному. В результате доля фильмов, оцененных по-разному мужчинами и женщинами, составляет 50/400, или 0,125. Это говорит нам о том, что 50/400 фильмов смотрятся по-разному мужчинами и женщинами.

#### Question #5:

#### Оценивают ли дети ,которые одни в семье фильм «Король Лев» (1994) выше, чем те,дети у кого есть братья и сестры?

- H0 : Единственным детям в семье фильм «Король Лев» (1994) нравится не больше, чем детям, у которых есть братья и сёстры.
- H1: Единственным детям в семье фильм «Король Лев» (1994) нравится больше, чем детям, у которых есть братья и сёстры.

In [ ]:
title = 'The Lion King (1994)'
data1.columns.get_loc('The Lion King (1994)') 

dataSib = data1.iloc[:,[220,475]] #476 - column for "only child"
                                 # " Are you an only child? (1: Yes; 0: No; -1: Did not respond)

## Разделение данных по признаку "единственный ребенок в семье"
notOnly =  dataSib.loc[dataSib['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 0 ]
onlyChild =  dataSib.loc[dataSib['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 1 ]

#разделение данных по признаку "единственный ребенок в семье" +  фильм "Король Лев (1994)" 
notOnlynp = np.array(notOnly['The Lion King (1994)'])
onlyChildnp = np.array(onlyChild['The Lion King (1994)'])

# отчистка от NaN
notOnlyprne = notOnlynp[np.isfinite(notOnlynp)]
onlyChildprne = onlyChildnp[np.isfinite(onlyChildnp)]

## тест Манна-Уитни U:
testmanSib = mannwhitneyu(notOnlyprne, onlyChildprne, use_continuity=True, alternative='less')

print(testmanSib)


MannwhitneyuResult(statistic=64247.0, pvalue=0.978419092554931)


#### Вывод:

В этом вопросе я применил односторонний тест Манна-Уитни, чтобы определить, существует ли значимая связь между удовольствием от фильма «Король Лев» у людей с братьями и сестрами и у тех, у кого нет братьев и сестер. В коде я разделил две интересующие меня группы на массивы и убедился, что ответы на вопрос «Вы единственный ребёнок в семье?» относятся к фильму «Король Лев». Затем интересующий меня набор данных был очищен поэлементно, чтобы снизить вероятность уменьшения размера выборки. Запуск теста как одностороннего позволяет тесту Манна-Уитни учитывать только группу, которой фильм понравился «больше». При p-значении 0,9784 мы не можем отвергнуть нулевую гипотезу с интервалом уровня альфа 0,005 и принимаем альтернативную гипотезу.

#### Вопрос #6:

#### Какая доля фильмов демонстрирует «эффект единственного ребенка», т. е. по-разному оценивается зрителями, у которых есть братья или сестры, и зрителями, у которых нет братьев или сестер?

In [13]:
all_titles = data1.columns
count = 0

for i in range(400):

    dataone = data1.loc[:,[all_titles[i], 'Are you an only child? (1: Yes; 0: No; -1: Did not respond)']]
    dataone = dataone[np.isfinite(dataone)]
    
    notonlyChild =  dataone.loc[dataone['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 0 ].iloc[:,0]
    onlyChild =  dataone.loc[dataone['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 1 ].iloc[:,0]
    #print(datafemale)
    notonlyChil = notonlyChild[np.isfinite(notonlyChild)]
    onlyChil = onlyChild[np.isfinite(onlyChild)]
   # print(datafem)
    test22, p = mannwhitneyu(notonlyChil, onlyChil)
    
    if p < 0.005:
        count += 1
proportion = count/400

print("Доля фильмов, в которых проявляется эффект 'единственного ребенка', т.е. которые оцениваются по-разному зрителями с братьями и сестрами и без них составляет ", proportion)

Доля фильмов, в которых проявляется эффект 'единственного ребенка', т.е. которые оцениваются по-разному зрителями с братьями и сестрами и без них составляет  0.0175


#### Вывод:

- Для этого вопроса я реализовал двухсторонний тест Манна-Уитни. Эта проблема была решена путем сначала отбора только интересующих данных, а затем категоризации моих данных на группу для участников, которые являются единственным ребенком в семье, и тех, у кого есть братья и сестры. Затем я очистил данные поэлементно, чтобы учесть количество данных как отдельных участников опроса, а не использовать очистку по строкам, где я потенциально могу сократить свои данные. Для расчета доли использовался U-тест Манна-Уитни с уровнем альфа 0,005, где с помощью цикла for достигалось p-значение для каждого элемента в строках и делилось на общий размер выборки, чтобы найти долю фильмов, которые продемонстрировали эффект единственного ребенка. Таким образом, мы можем заключить, что 7/400 фильмов демонстрируют «эффект единственного ребенка».

#### Question #7:

#### Нравится ли фильм «Волк с Уолл-стрит» (2013) тем, кто любит смотреть фильмы в компании, больше, чем тем, кто предпочитает смотреть его в одиночестве?

- H0 (Ноль): Людям, которые смотрят фильмы в компании, фильм «Волк с Уолл-стрит (2013)» нравится не больше, чем тем, кто предпочитает смотреть его в одиночестве.
- H1: Людям, которые смотрят фильмы в компании, фильм «Волк с Уолл-стрит (2013)» нравится больше, чем тем, кто предпочитает смотреть его в одиночестве.

In [15]:
##‘The Wolf of Wall Street (2013)’ -  358
## "Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)" -  477
title = 'The Wolf of Wall Street (2013)'
data1.columns.get_loc('The Wolf of Wall Street (2013)')
dataAlone = data1.iloc[:,[357,476]]    # [movie, question of interest]  
                
# деление данных на 2 группы интереса
notbestalone =  dataAlone.loc[dataAlone['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 0 ]
bestalone =  dataAlone.loc[dataAlone['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 1 ]

notbestalonenp = np.array(notbestalone['The Wolf of Wall Street (2013)'])
bestalonenp = np.array(bestalone['The Wolf of Wall Street (2013)'])

notbestaloneprne = notbestalonenp[np.isfinite(notbestalonenp)]
bestalonenpprne = bestalonenp[np.isfinite(bestalonenp)]

## Mann-Whitney U test  
testmanAlone = mannwhitneyu(notbestaloneprne, bestalonenpprne, use_continuity=True, alternative='greater')
print(testmanAlone)


MannwhitneyuResult(statistic=49303.5, pvalue=0.9436657996253056)


#### Вывод:

- Задача была решена с помощью одностороннего теста Манна-Уитни с уровнем альфа 0,005. Запуск одностороннего теста позволяет нам проверить влияние того, что одна группа получает больше удовольствия от фильма, чем другая. Перед обработкой данных я разделил выборку на две группы: «bestnotalone» и «bestalone». При этом я очистил данные поэлементно, чтобы исключить вероятность уменьшения размера выборки. В результате я получил p-значение 0,94366, что не позволяет отвергнуть нулевое значение с уровнем альфа 0,005.

#### Вопрос #8:

#### Какая доля фильмов демонстрирует подобный эффект «социального просмотра»?

In [16]:

all_titles = data1.columns
count = 0

for i in range(400):
    
    dataone = data1.loc[:,[all_titles[i], 'Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)']]
    dataone = dataone[np.isfinite(dataone)]
    
    notbestalone =  dataone.loc[dataAlone['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 0 ].iloc[:,0]
    bestalone =  dataone.loc[dataAlone['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 1 ].iloc[:,0]

    notbestalon = notbestalone[np.isfinite(notbestalone)]
    bestalon = bestalone[np.isfinite(bestalone)]
    test88, p = mannwhitneyu(notbestalon, bestalon, alternative='two-sided') 
    if p < 0.005:
        count += 1
        
proportion = count/400

print("пропорция фильмов, в которых проявляется эффект 'социального просмотра': " , proportion)

пропорция фильмов, в которых проявляется эффект 'социального просмотра':  0.025
